In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from lib.plots import load_data, climatology, single_column_initial_value

import seaborn as sns


width = 5.5
dpi = 100

sns.set_context("paper")
sns.set_style("white", {
    'figure.figsize' : (5.51, 5.51/1.61),
    'figure.dpi': 100,
    'image.cmap': 'viridis',
    'xtick.major.size': 3.5,
    'xtick.major.size': 3.5,
    'xtick.minor.size' : 1.75,
    'xtick.minor.visible' : True,
    'ytick.major.size': 3.5,
})


I put the styling commands in the following few boxes

In [ ]:
%%file paper.style

image.cmap : viridis
lines.linewidth : 1.0
savefig.bbox : tight
axes.labelsize      : small

xtick.labelsize : x-small
ytick.labelsize : x-small

In [ ]:
# plt.style.use(["default", "paper.style"])
width = 5.5
dpi = 100
# this magic needs to happen after I reset the rcParams
plt.rc("figure", dpi=150)

Here, I load the data and select the testing region.

In [ ]:
ds = load_data()
ds_test = ds.isel(x=slice(0,64))

# Figure 6: Forced Single Column

In [ ]:
import holoviews as hv
from holoviews.operation import contours
hv.extension('matplotlib')

In [ ]:

def plot_t_vs_p(loc, axs, cax=None,
    models = ('Truth', 'Neural Network', 'CAM'),
    levels = np.arange(11)*2, text_kwargs=dict(),
             **kwargs,):
    

    for model, ax in zip(models, axs):
        val = loc.sel(model=model)
        im = ax.contourf(val.time, val.p, val.T, levels=levels, **kwargs)
        ax.contour(val.time, val.p, val.T, levels=levels, linewidths=.1, **kwargs)
    
        # add labels
        plt.text(.04, .8, model, transform=ax.transAxes, fontsize=10, **text_kwargs)

#     axs[0].invert_yaxis()


#     plt.colorbar(im, orientation='vertical', pad=.01, **cbar_kwargs, )
    
    return im

In [ ]:
loc = ds.isel(x=10, y=8)

from matplotlib import gridspec


fig = plt.figure(figsize=(width, 6))
gs = gridspec.GridSpec(7, 2, width_ratios=(.95, .03),
                      height_ratios=[1]*7,
                       wspace=.05,
                      hspace=.1,
                      bottom=.07, top=.99)

# precip
axp = fig.add_subplot(gs[0, 0])
colors = {'Truth': 'k', 'CAM': 'g', 'Neural Network': 'b'}
for model, value in loc.groupby('model'):
    axp.plot(value.time, value.prec.squeeze(), label='model',
            linewidth=1.0, c=colors[model], alpha=.5)
    
for spine in set(axp.spines):
    axp.spines[spine].set_color('none')
axp.xaxis.set_visible(False)
axp.set_yticks([0, 50, 100])
axp.set_ylabel("mm/day")
axp.grid(axis='y')
axp.text(100, 110, 'Precipitation')
axp.set_xlim([ds.time.min(), ds.time.max()])

# humidity
axsq = [fig.add_subplot(gs[i, 0], sharex=axp) for i in range(1,4)]
imq = plot_t_vs_p(loc.qt, axsq, extend='min', text_kwargs={'color': 'white'})
cb = fig.colorbar(imq, cax=fig.add_subplot(gs[1:4, 1]))
cb.set_label(r"$q_T$ (g/kg)")

# temperature
axst = [fig.add_subplot(gs[i, 0], sharex=axp) for i in range(4,7)]
plot_t_vs_p(loc.sl, axst, extend='max', levels=np.arange(11)*5 + 290)
cb = fig.colorbar(imq, cax=fig.add_subplot(gs[4:, 1]))
cb.set_label(r"$s_L$ (K)")


# fix axes
for ax in axsq + axst:
    ax.set_ylabel('p (hPa)')
    ax.set_yticks([1000, 750, 500, 250])
    ax.set_ylim([1000, 0])
    
#     ax.set_xticklabels([1])
    
axp.set_xlim([100.625, 160])
axp.set_xticks(np.r_[105:160:5])

axst[-1].set_xlabel("time (days)")

plt.savefig("/Users/noah/Dropbox/fig6.pdf", bbox='tight')

# Figure 7:  Equilibrium

In [ ]:
with plt.rc_context({'axes.titlepad': 8.0}):
    climatology.plot(ds_test, width=width)
    plt.savefig("/Users/noah/Dropbox/fig7.pdf")